## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

You can get the data via:

    wget http://pjreddie.com/media/files/cifar.tgz

In [2]:
from fastai.conv_learner import *
PATH = "data/cifar10/"
os.makedirs(PATH,exist_ok=True)

In [3]:
classes = ('plane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [4]:
def get_data(sz,bs, partial_train_classes=None, max_train_per_class=None):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, partial_train_classes=partial_train_classes, max_train_per_class=max_train_per_class)

In [5]:
bs=256

## Fully connected model

In [6]:
data = get_data(32,bs)

In [7]:
lr=1e-2

From [this notebook](https://github.com/KeremTurgutlu/deeplearning/blob/master/Exploring%20Optimizers.ipynb) by our student Kerem Turgutlu:

## Refactored

In [8]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, kernel_size=3, stride=2, padding=1)
        
    def forward(self, x): return F.relu(self.conv(x))

## BatchNorm

In [9]:
class BnLayer(nn.Module):
    def __init__(self, ni, nf, stride=2, kernel_size=3):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, kernel_size=kernel_size, stride=stride,
                              bias=False, padding=1)
        self.a = nn.Parameter(torch.zeros(nf,1,1))
        self.m = nn.Parameter(torch.ones(nf,1,1))
        
    def forward(self, x):
        x = F.relu(self.conv(x))
        x_chan = x.transpose(0,1).contiguous().view(x.size(1), -1)
        if self.training:
            self.means = x_chan.mean(1)[:,None,None]
            self.stds  = x_chan.std (1)[:,None,None]
        return (x-self.means) / self.stds *self.m + self.a

## Resnet

In [10]:
class ResnetLayer(BnLayer):
    def forward(self, x): return x + super().forward(x)

## Resnet 2

In [11]:
class Resnet2(nn.Module):
    def __init__(self, layers, c, p=0.5):
        super().__init__()
        self.conv1 = BnLayer(3, 16, stride=1, kernel_size=7)
        self.layers = nn.ModuleList([BnLayer(layers[i], layers[i+1])
            for i in range(len(layers) - 1)])
        self.layers2 = nn.ModuleList([ResnetLayer(layers[i+1], layers[i + 1], 1)
            for i in range(len(layers) - 1)])
        self.layers3 = nn.ModuleList([ResnetLayer(layers[i+1], layers[i + 1], 1)
            for i in range(len(layers) - 1)])
        self.out = nn.Linear(layers[-1], c)
        self.drop = nn.Dropout(p)
        
    def forward(self, x):
        x = self.conv1(x)
        for l,l2,l3 in zip(self.layers, self.layers2, self.layers3):
            x = l3(l2(l(x)))
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(x.size(0), -1)
        x = self.drop(x)
        return F.log_softmax(self.out(x), dim=-1)

In [12]:
learn = ConvLearner.from_model_data(Resnet2([16, 32, 64, 128, 256], 10, 0.2), get_data(32,bs,partial_train_classes=None))

## Baseline

In [18]:
learn = ConvLearner.from_model_data(Resnet2([16, 32, 64, 128, 256], 10, 0.2), get_data(32,bs))
%time learn.fit(lr, 6)

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.740351   1.579138   0.4333    
    1      1.493747   1.31052    0.5332                     
    2      1.335309   1.315369   0.536                      
    3      1.229161   1.054946   0.6263                     
    4      1.086287   0.997337   0.6481                     
    5      1.007019   0.940983   0.6735                     

CPU times: user 1min 36s, sys: 43 s, total: 2min 19s
Wall time: 1min 17s


[array([0.94098]), 0.6735]

After seeing 6 times the number of images in the dataset (6 epochs), the network obtained an accuracy of 0.67


## Jeremy

In [13]:
learn = ConvLearner.from_model_data(Resnet2([16, 32, 64, 128, 256], 10, 0.2), get_data(32,bs))
learn.set_data(get_data(32,bs,partial_train_classes=range(2)))
learn.unfreeze()
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.452692   35.176036  0.1781    
    1      0.305815   23.294715  0.185                     



[array([23.29472]), 0.185]

In [14]:
learn.set_data(get_data(32,bs,partial_train_classes=range(4)))
learn.unfreeze()
%time learn.fit(lr, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.274154   33.665933  0.2341    
    1      0.951999   24.299984  0.252                     

CPU times: user 14.9 s, sys: 7.07 s, total: 22 s
Wall time: 11.8 s


[array([24.29998]), 0.252]

In [15]:
learn.set_data(get_data(32,bs,partial_train_classes=range(6)))
learn.unfreeze()
%time learn.fit(lr, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.407722   4.658204   0.3096    
    1      1.237811   5.112041   0.3242                     

CPU times: user 21 s, sys: 9.08 s, total: 30.1 s
Wall time: 16.5 s


[array([5.11204]), 0.3242]

In [16]:
learn.set_data(get_data(32,bs,partial_train_classes=range(8)))
learn.unfreeze()
%time learn.fit(lr, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.402549   6.337645   0.4243    
    1      1.254833   3.832431   0.4638                     

CPU times: user 26 s, sys: 12.1 s, total: 38.1 s
Wall time: 20.9 s


[array([3.83243]), 0.4638]

In [17]:
learn.set_data(get_data(32,bs))
learn.unfreeze()
%time learn.fit(lr, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.44822    1.266059   0.5433    
    1      1.305428   1.151673   0.5861                     

CPU times: user 33 s, sys: 14.4 s, total: 47.4 s
Wall time: 26.2 s


[array([1.15167]), 0.5861]

After seeing 6 times the number of images in the dataset ((0.2+0.4+0.6+0.8+1)*2=6), the network obtained an accuracy of 0.58

## Leslie

In [29]:
n_trn = data.trn_ds.n
n_classes = data.trn_ds.c

In [32]:
learn = ConvLearner.from_model_data(Resnet2([16, 32, 64, 128, 256], 10, 0.2), get_data(32,bs))
learn.set_data(get_data(32,bs,max_train_per_class=int(0.2*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      2.288043   15.790987  0.1431    
    1      2.182858   3.760559   0.3335                   



[array([3.76056]), 0.3335]

In [33]:
learn.set_data(get_data(32,bs,max_train_per_class=int(0.4*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.731712   1.577297   0.4476    
    1      1.692346   1.856433   0.4046                   



[array([1.85643]), 0.4046]

In [34]:
learn.set_data(get_data(32,bs,max_train_per_class=int(0.6*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.539406   1.77725    0.4809    
    1      1.466001   1.431418   0.5226                     



[array([1.43142]), 0.5226]

In [35]:
learn.set_data(get_data(32,bs,max_train_per_class=int(0.8*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.285457   1.297473   0.5676    
    1      1.235451   1.164417   0.5934                     



[array([1.16442]), 0.5934]

In [36]:
learn.set_data(get_data(32,bs,max_train_per_class=int(1*(n_trn/n_classes))))
learn.unfreeze()
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.095149   1.033715   0.6452    
    1      1.018922   0.951017   0.6663                     



[array([0.95102]), 0.6663]

## Verdict

1st place - Baseline - 67%<br/>
2nd place - Leslie - 66.6%<br/>
3rd place - Jeremy - 58%<br/>
